In [1]:
import pandas as pd
import numpy as np
import requests
import io
from sklearn.model_selection import TimeSeriesSplit
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from lime.lime_tabular import LimeTabularExplainer

# import pyreadr module
import pyreadr


In [2]:
def get_energy_demand(scale: bool = True) -> pd.DataFrame: 
    resp = requests.get( 
        "https://github.com/camroach87/gefcom2017data/raw/master/data/gefcom.rda", 
        allow_redirects=True, 
    ) 
    open("gefcom.rda", "wb").write(resp.content) 
    result = pyreadr.read_r("gefcom.rda") 
    df = result["gefcom"].pivot(index="ts", columns="zone", values="demand") 
    df = df.asfreq("D") 
    if not scale: 
        return df 
    return pd.DataFrame( 
        data=StandardScaler().fit_transform(df), columns=df.columns, index=df.index 
    ) 
 


In [6]:


# (a) Download and clean up the Connecticut time series energy data

energy_data = get_energy_demand(scale=False)
print(energy_data.head(2))
energy_data = energy_data[['CT']]
energy_data = energy_data.loc[energy_data.index.year == 2017]


zone            CT    MASS      ME  NEMASSBOST      NH     RI  SEMASS  \
ts                                                                      
2003-03-01  3386.0  5913.0  1111.0      2574.0  1055.0  791.0  1484.0   
2003-03-02  3122.0  5449.0  1033.0      2407.0   963.0  737.0  1357.0   

zone          TOTAL     VT  WCMASS  
ts                                  
2003-03-01  12864.0  608.0  1855.0  
2003-03-02  11862.0  558.0  1685.0  


In [ ]:

# (b) Download and clean up 2 suitable exogenous variables
# For this example, I'll use historical weather data from the National Oceanic and Atmospheric Administration (NOAA)
# We'll use the average temperature and precipitation data for Hartford, CT

# You can replace the API key with your own key from https://www.ncdc.noaa.gov/cdo-web/webservices/v2
API_KEY = 'your_api_key_here'
start_date = '2017-01-01'
end_date = '2017-12-31'
station_id = 'GHCND:USW00014740'  # Weather station ID for Hartford, CT

url = f'https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&dataTypes=TAVG,PRCP&stations={station_id}&startDate={start_date}&endDate={end_date}&includeAttributes=true&format=csv&units=standard&limit=1000&apiKey={API_KEY}'
response = requests.get(url)
weather_data = pd.read_csv(io.StringIO(response.text), parse_dates=['DATE'], index_col='DATE')
weather_data = weather_data[['TAVG', 'PRCP']]


In [ ]:

# (c) Combine the two datasets and create column features for week, month, year
combined_data = energy_data.join(weather_data)
combined_data['Year'] = combined_data.index.year
combined_data['Month'] = combined_data.index.month
combined_data['Week'] = combined_data.index.week
combined_data['Day'] = combined_data.index.day


In [ ]:

# Split into train and test (80/20, not shuffled)
tscv = TimeSeriesSplit(n_splits=2)
for train_index, test_index in tscv.split(combined_data):
    train_data = combined_data.iloc[train_index]
    test_data = combined_data.iloc[test_index]


In [ ]:

# (d) Initialize an XGBoost model for time series (can fine tune learning rate, max depth, etc.)
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=1000, learning_rate=0.1, max_depth=5, subsample=0.8, colsample_bytree=0.8, n_jobs=-1)


In [ ]:

# (e) Train the resulting model on a multivariate problem
X_train, y_train = train_data.drop('Consumption', axis=1), train_data['Consumption']
X_test, y_test = test_data.drop('Consumption', axis=1), test_data['Consumption']
model.fit(X_train, y_train)




In [ ]:

# (f) Plot the time series prediction with confidence intervals
y_pred = model.predict(X_test)
confidence_interval = 1.96 * np.std(y_pred) / np.mean(y_pred)

fig, ax = plt.subplots(figsize=(12, 6))
sns.lineplot(x=test_data.index, y=y_test, label='Ground truth', ax=ax)
sns.lineplot(x=test_data.index, y=y_pred, label='Prediction', ax=ax)
ax.fill_between(test_data.index, (y_pred - confidence_interval), (y_pred + confidence_interval), color='b', alpha=.1, label='Confidence interval')

plt.title('XGBoost time series prediction')
plt.xlabel('Date')
plt.ylabel('Energy consumption')
plt.legend()
plt.show()


In [ ]:

# (g) Perform feature importance using the LIME library
explainer = LimeTabularExplainer(X_train.values,
                                 feature_names=X_train.columns,
                                 class_names=['Consumption'],
                                 mode='regression')

# Select a specific instance for explanation
instance = X_test.iloc[10].values
exp = explainer.explain_instance(instance, model.predict, num_features=len(X_train.columns))

# Show relative importance of features
exp.show_in_notebook(show_table=True)

# Show the local linear approximation of the model
exp.as_pyplot_figure()

